## Q1

In [178]:
#Q1 A
import numpy as np

# Funtion to move the rows with all zeros to bottom
def Move_rows_with_all_zeros(matrix):
    num_rows = len(matrix)
    num_cols = len(matrix[0])

    # Separate rows with all zeros and non-zero rows
    all_zeros_rows = [row for row in matrix if all(element == 0 for element in row[:-1])]
    non_zero_rows = [row for row in matrix if any(element != 0 for element in row[:-1])]

    # Keep all zero rows at bottom
    result_matrix = non_zero_rows + all_zeros_rows

    return result_matrix

# Funtion to swap the rows in row operations
def Swap_rows(matrix, i, j):
    # Swap rows i and j in the matrix
    matrix[i], matrix[j] = matrix[j], matrix[i]

# Funtion to swap the rows in row operations    
def Scale_row(matrix, i, scale):
    # Multiply the entire ith row by a scalar
    matrix[i] = [entry * scale for entry in matrix[i]]

# Funtion to add scaled row in row operations
def Add_scaled_row(matrix, i, j, scale):
    # Add a scaled version of row j to row i
    matrix[i] = [entry_i + scale * entry_j for entry_i, entry_j in zip(matrix[i], matrix[j])]

# Function to print matrix    
def Print_matrix(matrix):
    for row in matrix:
        print([0 if entry == 0 else entry for entry in row])

# Function to get the Row echelon form of given A, B matrices
def Get_Row_Echelon_Form(A, B):
    
    # Combine matrices A and B to form the augmented matrix AB
    AB = []

    for i in range(len(A)):
        row_AB = list(A[i]) + list(B[i])
        AB.append(row_AB)
    
    matrix = np.array(AB, dtype=np.float64)
    
    num_rows, num_cols = matrix.shape

    # Perform row operations to get row echelon form
    for i in range(min(num_rows, num_cols - 1)):
        
        # Find the first nonzero row and scale it to have a leading 1
        nonzero_row = next((row for row in range(i, num_rows) if matrix[row][i] != 0), None)
        if nonzero_row is not None:
            swap_rows(matrix, i, nonzero_row)
            pivot_value = matrix[i][i]

            # Check for division by zero
            if pivot_value == 0:
                print("Error: Division by zero. Please provide a different input matrix.")
                return matrix

            scale_row(matrix, i, 1 / pivot_value)

            # Eliminate other entries in the current column
            for j in range(i + 1, num_rows):
                add_scaled_row(matrix, j, i, -matrix[j][i])
                
    matrix = move_rows_with_all_zeros(matrix)

    return matrix

# Function to get the Reduced Row echelon form of given A, B matrices
def Get_Reduced_Row_Echelon_Form(A, B):
    
    matrix =  np.array(row_echelon(A, B))
    num_rows, num_cols = matrix.shape

    # Make every pivot element 1
    for i in range(num_rows):
        pivot_col = next((col for col in range(num_cols - 1) if matrix[i][col] != 0), None)
        if pivot_col is not None:
            pivot_val = matrix[i][pivot_col]
            scale_row(matrix, i, 1 / pivot_val)

            # Make the pivot element the only nonzero entry in its column
            for j in range(num_rows):
                if j != i:
                    add_scaled_row(matrix, j, i, -matrix[j][pivot_col])

    return matrix

In [180]:
#Q1 B
# Function to get pivot and non-zero pivot columns
def Get_pivot_nonpivot_columns(matrix):
    num_rows = len(matrix)
    num_cols = len(matrix[0])

    pivot_columns = []
    nonpivot_columns = []

    for row in matrix:
        found_pivot = False
        for col_index, value in enumerate(row[:-1]):  # Exclude the last column
            if value != 0 and not found_pivot:
                pivot_columns.append(col_index)
                found_pivot = True
            elif value != 0 and found_pivot:
                nonpivot_columns.append(col_index)

    # Remove duplicate columns
    pivot_columns = list(set(pivot_columns))
    nonpivot_columns = list(set(nonpivot_columns))
    
    return pivot_columns, nonpivot_columns

# Function to get particular solution of a reduced row echelon matrix
def Find_particular_solution(row_echelon_form):
    pivot_columns, _ = identify_pivot_nonpivot_columns(row_echelon_form)

    particular_solution = [0] * len(pivot_columns)

    for i in range(len(pivot_columns)):
        col_index = pivot_columns[i]
        pivot_row = next((row_index for row_index, value in enumerate(row_echelon_form) if value[col_index] != 0), None)
        if pivot_row is not None:
            particular_solution[i] = row_echelon_form[pivot_row][-1]

    variable_names = [f'x{i+1}' for i in pivot_columns]
    return dict(zip(variable_names, particular_solution))

# Function to get general solution of a reduced row echelon matrix
def Find_general_solution(row_echelon_form):
    _, nonpivot_columns = identify_pivot_nonpivot_columns(row_echelon_form)
    num_variables = len(row_echelon_form[0]) - 1

    general_solution_coefficients = []
    
    for col_index in nonpivot_columns:
        coefficients = [0] * (num_variables)  # Extend to include the constant term
        pivot_columns_left = [i for i in range(col_index) if i not in nonpivot_columns]

        for pivot_col_index in pivot_columns_left:
            pivot_row = next((row_index for row_index, value in enumerate(row_echelon_form) if value[pivot_col_index] != 0), None)
            coefficients[pivot_col_index] = -row_echelon_form[pivot_row][col_index]

        coefficients[col_index] = 1
        general_solution_coefficients.append(coefficients)

    return np.array(general_solution_coefficients)

# Function to get solutions to a given A, B matrices
def Find_Solutions_For_Linear_System(row_echelon_form):
    
    Pivot_columns = Get_pivot_nonpivot_columns(row_echelon_form)[0]
    Non_Pivot_columns = Get_pivot_nonpivot_columns(row_echelon_form)[1]
    print('Pivot columns are: ',Pivot_columns)
    print('Non_Pivot_columns are: ',Non_Pivot_columns)
    
    # Calculate the rank of A and AB
    # Separate A and B from the augmented matrix
    A = [row[:-1] for row in row_echelon_form]
    B = [row[-1] for row in row_echelon_form]
    
    # Calculate the rank of A and AB
    rank_A = sum(1 for row in A if any(element != 0 for element in row))
    rank_AB = sum(1 for row in row_echelon_form if any(element != 0 for element in row))
    
    # check the consistency of the system
    if rank_A < rank_AB:
        print("The system is inconsistent.")
    elif rank_A == rank_AB:
        print("The system is consistent.")
        free_variables_count = len(Non_Pivot_columns)
        
        if rank_A == free_variables_count:
            print("The system has a unique solution.")
            Particular_solution = Find_particular_solution(row_echelon_form)
            print("The particular solutions is: ",Particular_solution)
        else:
            print("The system has infinitely many solutions.")
            Particular_solution = Find_particular_solution(row_echelon_form)
            print("The particular solutions is: ",Particular_solution)
            General_solution = Find_general_solution(row_echelon_form)
            print("The General solutions is: ",General_solution)


In [181]:
#Q1 C
import numpy as np

# Generate a random 5x7 matrix for A
A = np.random.rand(5, 7)

# Generate a random b vector matrix
b = np.random.rand(5, 1)

# Round off to 2 decimals
A = np.round(A, decimals=2)
B = np.round(b, decimals=2)

print("Matrix A:")
print(A_rounded)

print("\nVector b:")
print(b_rounded)


Matrix A:
[[0.37 0.63 0.63 0.54 0.09 0.84 0.32]
 [0.19 0.04 0.59 0.68 0.02 0.51 0.23]
 [0.65 0.17 0.69 0.39 0.94 0.14 0.34]
 [0.11 0.92 0.88 0.26 0.66 0.82 0.56]
 [0.53 0.24 0.09 0.9  0.9  0.63 0.34]]

Vector b:
[[0.35]
 [0.73]
 [0.9 ]
 [0.89]
 [0.78]]


In [182]:
Get_Row_Echelon_Form(A,B)

[array([1.        , 0.38202247, 0.42696629, 0.1011236 , 0.65168539,
        0.04494382, 0.52808989, 0.78651685]),
 array([ 0.        ,  1.        ,  4.29395973, -0.29395973, -3.7261745 ,
         9.50604027,  0.89395973, -0.17583893]),
 array([-0.        , -0.        ,  1.        , -0.26443213, -1.75298002,
         3.36130452,  0.33645648, -0.11528216]),
 array([ 0.        ,  0.        ,  0.        ,  1.        ,  0.15319082,
         1.53191598,  0.17302345, -0.63154667]),
 array([-0.        , -0.        , -0.        , -0.        ,  1.        ,
        -1.7371387 , -1.85163669,  1.67671353])]

In [183]:
Get_Reduced_Row_Echelon_Form(A,B)

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.59507221,  0.54773829,  0.70543601],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.16264288,  6.10310047, -5.30650296],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.79159191, -2.78866552,  2.58904065],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         1.79802968,  0.45667719, -0.88840379],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        -1.7371387 , -1.85163669,  1.67671353]])

In [184]:
Find_Solutions_For_Linear_System(Get_Reduced_Row_Echelon_Form(A, B))

Pivot columns are:  [0, 1, 2, 3, 4]
Non_Pivot_columns are:  [5, 6]
The system is consistent.
The system has infinitely many solutions.
The particular solutions is:  {'x1': 0.7054360126294027, 'x2': -5.306502960407149, 'x3': 2.589040651004763, 'x4': -0.8884037902561184, 'x5': 1.6767135328267302}
The General solutions is:  [[-0.59507221 -0.16264288 -0.79159191 -1.79802968  1.7371387   1.
   0.        ]
 [-0.54773829 -6.10310047  2.78866552 -0.45667719  1.85163669  0.
   1.        ]]


In [186]:
# Own example
A = np.array([[-2,4,-2,-1,4], [4,-8,3,-3,1], [1,-2,1,-1,1], [1,-2,0,-3,4]], dtype=np.float64)
B = np.array([[-3],[2],[0],[-1]], dtype=np.float64)

# Convert to row echelon form
Row_echelon_form = row_echelon(A, B)
print("Row Echelon Form:")
print_matrix(row_echelon_form)

# Convert to reduced row echelon form
reduced_row_echelon_form = v1_reduced_row_echelon(A, B)
print("\nReduced Row Echelon Form:")
print_matrix(reduced_row_echelon_form)


Row Echelon Form:
[1.0, -2.0, 1.0, 0.5, -2.0, 1.5]
[0, 0, -1.0, -5.0, 9.0, -4.0]
[0, 0, 1.0, 3.5, -6.0, 2.5]
[0, 0, 0, 0, 0, 0]

Reduced Row Echelon Form:
[1.0, -2.0, 0, 0, -2.0, 2.0]
[0, 0, 1.0, 0, 1.0, -1.0]
[0, 0, 0, 1.0, -2.0, 1.0]
[0, 0, 0, 0, 0, 0]


In [187]:
Get_Reduced_Row_Echelon_Form(A,B)

array([[ 1., -2.,  0.,  0., -2.,  2.],
       [ 0.,  0.,  1.,  0.,  1., -1.],
       [-0., -0., -0.,  1., -2.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [188]:
Get_Reduced_Row_Echelon_Form(A,B)

array([[ 1., -2.,  0.,  0., -2.,  2.],
       [ 0.,  0.,  1.,  0.,  1., -1.],
       [-0., -0., -0.,  1., -2.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [189]:
Find_Solutions_For_Linear_System(Get_Reduced_Row_Echelon_Form(A, B))

Pivot columns are:  [0, 2, 3]
Non_Pivot_columns are:  [1, 4]
The system is consistent.
The system has infinitely many solutions.
The particular solutions is:  {'x1': 2.0, 'x3': -1.0, 'x4': 1.0}
The General solutions is:  [[ 2.  1.  0.  0.  0.]
 [ 2.  0. -1.  2.  1.]]


## Q2

In [192]:
# Q2 a
def Matrix_Decomposition_LU(A):
    
    # Checking if the matrix is square
    if not all(len(row) == len(A) for row in A):
        print("Error: The matrix is not square.")
        return

    # Checking if the matrix is symmetric
    if not all(A[i][j] == A[j][i] for i in range(len(A)) for j in range(len(A))):
        print("Error: The matrix is not symmetric.")
        return

    # Perform LU decomposition of input A matrix
    n = len(A)
    L = [[0] * n for _ in range(n)]
    U = [[0] * n for _ in range(n)]

    for i in range(n):
        # Get the Upper Triangular matrix 
        for k in range(i, n):
            sum_ = sum(L[i][p] * U[p][k] for p in range(i))
            U[i][k] = A[i][k] - sum_

        # Get the Lower Triangular matrix
        for k in range(i, n):
            if i == k:
                L[i][i] = 1  # Diagonal is 1
            else:
                sum_ = sum(L[k][p] * U[p][i] for p in range(i))
                L[k][i] = (A[k][i] - sum_) / U[i][i]

    # Display the input matrix
    print("Matrix A:")
    for row in A:
        print(row)

    print("\nLower Triangular Matrix L:")
    for row in L:
        print(row)

    print("\nUpper Triangular Matrix U:")
    for row in U:
        print(row)

    # Verify A = LU
    A_reconstructed = [[sum(L[i][p] * U[p][k] for p in range(len(L[0]))) for k in range(len(U[0]))] for i in range(len(L))]
    print("\nReconstructed Matrix A from LU:")
    for row in A_reconstructed:
        print(row)

In [193]:
# Example usage
A = np.array([[25, 15, -5],
              [15, 18, 0],
              [-5, 0, 11]])
Matrix_Decomposition_LU(A)

Matrix A:
[25 15 -5]
[15 18  0]
[-5  0 11]

Lower Triangular Matrix L:
[1, 0, 0]
[0.6, 1, 0]
[-0.2, 0.3333333333333333, 1]

Upper Triangular Matrix U:
[25, 15, -5]
[0, 9.0, 3.0]
[0, 0, 9.0]

Reconstructed Matrix A from LU:
[25, 15.0, -5.0]
[15.0, 18.0, 0.0]
[-5.0, 0.0, 11.0]


In [194]:
# Q2 b
import numpy as np

def matrix_multiply(A, B):
    
    rows_A, cols_A = len(A), len(A[0])
    rows_B, cols_B = len(B), len(B[0])

    result = [[0] * cols_B for _ in range(rows_A)]

    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                result[i][j] += A[i][k] * B[k][j]

    return result

def transpose_matrix(matrix):
    rows, cols = len(matrix), len(matrix[0])
    result = [[0] * rows for _ in range(cols)]

    for i in range(rows):
        for j in range(cols):
            result[j][i] = matrix[i][j]

    return result

def cholesky_decomposition(matrix):
      
    # Checking if the matrix is square
    if not all(len(row) == len(matrix) for row in matrix):
        print("Error: The matrix is not square.")
        return

    # Checking if the matrix is symmetric
    if not all(matrix[i][j] == matrix[j][i] for i in range(len(matrix)) for j in range(len(matrix))):
        print("Error: The matrix is not symmetric.")
        return
    
    n = len(matrix)
    L = np.zeros((n, n))
    
    # Get the lower triangular matrix from input matrix
    for i in range(n):
        for j in range(i + 1):
            sum_val = sum(L[i][k] * L[j][k] for k in range(j))
            
            if i == j:
                L[i][j] = np.sqrt(matrix[i][i] - sum_val)
            else:
                L[i][j] = (1.0 / L[j][j] * (matrix[i][j] - sum_val))
                
    # Get the transpose of the lower triangular matrix
    LT = transpose_matrix(L)
    
    # Get the reconstructed matrix by L * LT
    reconstructed_A = matrix_multiply(L, LT)
    
    print("\nLower Triangular Matrix L:")
    print(L)

    print("\nTranspose of Lower Triangular Matrix LT:")
    print(LT)

    print("\nReconstructed Matrix A from LL^T:")
    print(reconstructed_A)

    print("\nOriginal Matrix A:")
    print(matrix)
   
    if(np.array_equal(matrix, reconstructed_A)):
        print('Yes, it satifies cholesky_decomposition ')

In [195]:
cholesky_decomposition(A)


Lower Triangular Matrix L:
[[ 5.  0.  0.]
 [ 3.  3.  0.]
 [-1.  1.  3.]]

Transpose of Lower Triangular Matrix LT:
[[5.0, 3.0, -1.0], [0.0, 3.0, 1.0], [0.0, 0.0, 3.0]]

Reconstructed Matrix A from LL^T:
[[25.0, 15.0, -5.0], [15.0, 18.0, 0.0], [-5.0, 0.0, 11.0]]

Original Matrix A:
[[25 15 -5]
 [15 18  0]
 [-5  0 11]]
Yes, it satifies cholesky_decomposition 


In [196]:
# Q2 c
import numpy as np

def matrix_multiply(A, B):
    rows_A, cols_A = len(A), len(A[0])
    rows_B, cols_B = len(B), len(B[0])

    if cols_A != rows_B:
        raise ValueError("Number of columns in A must be equal to the number of rows in B for matrix multiplication.")

    result = [[0] * cols_B for _ in range(rows_A)]

    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                result[i][j] += A[i][k] * B[k][j]

    return result

def transpose_matrix(matrix):
    rows, cols = len(matrix), len(matrix[0])
    result = [[0] * rows for _ in range(cols)]

    for i in range(rows):
        for j in range(cols):
            result[j][i] = matrix[i][j]

    return result

def QR_decomposition(A):
    m, n = A.shape

    # Initialize Q and R matrices
    Q = np.zeros((m, n))
    R = np.zeros((n, n))

    for j in range(n):
        # Orthogonalization of matrix
        v = A[:, j].copy()
        for i in range(j):
            R[i, j] = np.dot(Q[:, i], A[:, j])
            v -= R[i, j] * Q[:, i]

        # Normalization of matrix
        norm_v = np.linalg.norm(v)
        if norm_v < 1e-8:
            # Handling the case where v is very close to a zero vector
            Q[:, j] = 0.0
        else:
            Q[:, j] = v / norm_v
        R[j, j] = norm_v

    # Validating QR decomposition
    A_reconstructed = matrix_multiply(Q, R)
    print("\nReconstructed Matrix A from QR:")
    print(np.round(A_reconstructed, decimals=2))
    
    # Resconstructing R from QT, A
    R_reconstructed = matrix_multiply(transpose_matrix(Q), A)
    print("\nReconstructed Matrix R from Q^T * A:")
    print(np.round(R_reconstructed, decimals=2))

    print("\nValidation check: R == Q^T * A")
    print('Is it satisfying: ',np.allclose(R, R_reconstructed))

    # Display Q and R
    print("\nMatrix Q:")
    print(np.round(Q, decimals=2))

    print("\nMatrix R:")
    print(np.round(R, decimals=2))

In [197]:
# With sample example
A = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]], dtype=float)
QR_decomposition(A)


Reconstructed Matrix A from QR:
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]

Reconstructed Matrix R from Q^T * A:
[[ 8.12  9.6  11.08]
 [ 0.    0.9   1.81]
 [ 0.    0.    0.  ]]

Validation check: R == Q^T * A
Is it satisfying:  True

Matrix Q:
[[ 0.12  0.9   0.  ]
 [ 0.49  0.3   0.  ]
 [ 0.86 -0.3   0.  ]]

Matrix R:
[[ 8.12  9.6  11.08]
 [ 0.    0.9   1.81]
 [ 0.    0.    0.  ]]


In [198]:
# Q2 d
#Q2 D
import numpy as np

# Generate a random 5x4 matrix with linearly independent columns
random_matrix = np.random.rand(5, 4)

# Round off to 2 decimals
random_matrix = np.round(random_matrix, decimals=2)

# Display the rounded random matrix
print(random_matrix)


[[0.55 0.71 0.66 0.28]
 [0.95 0.74 0.55 0.61]
 [0.42 0.25 0.36 0.76]
 [0.01 0.12 0.05 0.04]
 [0.86 0.7  0.47 0.1 ]]


In [199]:
QR_decomposition(random_matrix)


Reconstructed Matrix A from QR:
[[0.55 0.71 0.66 0.28]
 [0.95 0.74 0.55 0.61]
 [0.42 0.25 0.36 0.76]
 [0.01 0.12 0.05 0.04]
 [0.86 0.7  0.47 0.1 ]]

Reconstructed Matrix R from Q^T * A:
[[ 1.46  1.24  0.99  0.78]
 [ 0.    0.3   0.26 -0.18]
 [-0.   -0.    0.21  0.53]
 [ 0.    0.    0.    0.33]]

Validation check: R == Q^T * A
Is it satisfying:  True

Matrix Q:
[[ 0.38  0.82  0.36 -0.19]
 [ 0.65 -0.23 -0.18  0.46]
 [ 0.29 -0.36  0.78  0.16]
 [ 0.01  0.38 -0.25  0.7 ]
 [ 0.59 -0.1  -0.41 -0.48]]

Matrix R:
[[ 1.46  1.24  0.99  0.78]
 [ 0.    0.3   0.26 -0.18]
 [ 0.    0.    0.21  0.53]
 [ 0.    0.    0.    0.33]]
